# Pre-process BEST data
Downloaded BEST data is concatenated into a single file and subset to being limited to the continental United States. 

## Setup

In [1]:
import xarray as xr
import numpy as np
import calendar
import datetime as dt

In [2]:
output_fn = '../data/climate_data/tas_day_BEST_historical_station_19800101-20091231.nc'

### Set geographic subset
Using the extreme points of the continental United States (see e.g. [here](https://en.wikipedia.org/wiki/List_of_extreme_points_of_the_United_States)) + 1 degree of wiggle room.

In [3]:
geo_lims = {'lat':[23,51],'lon':[-126,-65]}

## Load files

In [4]:
ds = xr.open_mfdataset('../data/climate_data/Complete*.nc',combine='nested',concat_dim='time')
ds

<xarray.Dataset>
Dimensions:      (day_number: 365, latitude: 180, longitude: 360, time: 10958)
Coordinates:
  * longitude    (longitude) float32 -179.5 -178.5 -177.5 ... 177.5 178.5 179.5
  * latitude     (latitude) float32 -89.5 -88.5 -87.5 -86.5 ... 87.5 88.5 89.5
Dimensions without coordinates: day_number, time
Data variables:
    date_number  (time) float64 dask.array<chunksize=(3653,), meta=np.ndarray>
    year         (time) float64 dask.array<chunksize=(3653,), meta=np.ndarray>
    month        (time) float64 dask.array<chunksize=(3653,), meta=np.ndarray>
    day          (time) float64 dask.array<chunksize=(3653,), meta=np.ndarray>
    day_of_year  (time) float64 dask.array<chunksize=(3653,), meta=np.ndarray>
    land_mask    (time, latitude, longitude) float64 dask.array<chunksize=(3653, 180, 360), meta=np.ndarray>
    temperature  (time, latitude, longitude) float32 dask.array<chunksize=(3653, 180, 360), meta=np.ndarray>
    climatology  (time, day_number, latitude, longitude) float32 dask.array<chunksize=(3653, 365, 180, 360), meta=np.ndarray>
Attributes:
    Conventions:     Berkeley Earth Internal Convention (based on CF-1.5)
    title:           Gridded Berkeley Earth Surface Temperature Anomaly Field
    history:         17-Jun-2018 21:11:44
    institution:     Berkeley Earth Surface Temperature Project
    source_history:  17-Jun-2018 16:15:15
    comment:         This file contains surface temperature anomaly field gri...

In [5]:
# Through the concat_dim, some variables were broadcast over dimensions they shouldn't have been
ds['climatology'] = ds.climatology.isel(time=0)
ds['land_mask'] = ds.land_mask.isel(time=0)

In [6]:
# Subset geographically
ds = ds.sel(latitude=slice(*geo_lims['lat']),longitude=slice(*geo_lims['lon'])).load()

In [7]:
# Create time variable, which wasn't auto-generated from the netcdf due to BEST's ambiguous timing
ds['time'] = (('time'),dt.datetime(1980,1,1)+np.arange(0,ds.dims['time'])*dt.timedelta(days=1))

### Create temperature from anomaly + climatology 
The variable `temperature` in BEST is actually the temperature _anomaly_; the actual temperature is formed by adding it to the `climatology` variable. Unfortunately, the `climatology` variable only accounts for days 1:365 of the year, and ignores leap days (which the `temperature` variable does not). This section doubles the climatology for Feb 28th to also work on Feb 29th, and creates a `tas` variable that's the `climatology` + `temperature`. 

In [8]:
# Expand climatology to span all days
clim_tmp = xr.DataArray(dims=('time','latitude','longitude'),
                        coords={'time':ds.time,'latitude':ds.latitude,'longitude':ds.longitude},
                        data=np.zeros((ds.dims['time'],ds.dims['latitude'],ds.dims['longitude']))*np.nan)

# Sub in variables one year at a time
for yr in np.unique(ds.time.dt.year):
    if calendar.isleap(yr):
        clim_tmp.loc[{'time':(clim_tmp.time.dt.year==yr)&(clim_tmp.time.dt.dayofyear<=59)}] = ds.climatology.values[0:59]
        clim_tmp.loc[{'time':(clim_tmp.time.dt.year==yr)&(clim_tmp.time.dt.dayofyear==60)}] = ds.climatology.values[59]
        clim_tmp.loc[{'time':(clim_tmp.time.dt.year==yr)&(clim_tmp.time.dt.dayofyear>60)}] = ds.climatology.values[59:365]
    else:
        clim_tmp.loc[{'time':(clim_tmp.time.dt.year==yr)}] = ds.climatology.values

In [9]:
ds['climatology'] = clim_tmp

In [10]:
ds['tas'] = ds['temperature'] + ds['climatology']
ds = ds.drop(['temperature','climatology'])

## Save

In [11]:
ds.attrs['origin_script']='preprocess_best.ipynb'

In [13]:
# Rename to lat/lon to fit CMIP5 defaults
ds = ds.rename({'latitude':'lat','longitude':'lon'})

In [15]:
ds.to_netcdf(output_fn)